## Predict the Diabetes Retinopathy (DR) by using Ensemble method
### Using the features from image to predict the patients whether have DR

In [1]:
# Import the functions
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import pandas as pd
import statistics as stat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pydotplus
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image
from statistics import mode 

In [2]:
# Change the working directory
import os
os.chdir("/Users/SK/Desktop/SK/NUS EBA/Semester 4/Advance Analytic Project/Diabetic Retinopathy")

In [3]:
# Read the CSV file
df = pd.read_csv("train.csv")

In [4]:
# Explore the data
df.shape

(920, 20)

In [5]:
df.dtypes

q            int64
ps           int64
nma.a        int64
nma.b        int64
nma.c        int64
nma.d        int64
nma.e        int64
nma.f        int64
nex.a      float64
nex.b      float64
nex.c      float64
nex.d      float64
nex.e      float64
nma.f.1    float64
nma.g      float64
nma.h      float64
dd         float64
dm         float64
amfm         int64
Class        int64
dtype: object

In [6]:
df.head()

,q,ps,nma.a,nma.b,nma.c,nma.d,nma.e,nma.f,nex.a,nex.b,nex.c,nex.d,nex.e,nma.f.1,nma.g,nma.h,dd,dm,amfm,Class
0,1,1,75,63,60,55,48,35,13.195493,4.396967,0.104070,0.000000,0.000000,0.000000,0.000000,0.000000,0.513092,0.123966,0,1
1,1,1,79,76,74,72,69,50,61.559348,28.959444,12.778104,2.045287,0.038016,0.000000,0.000000,0.000000,0.527993,0.101884,0,1
2,1,1,41,41,40,40,38,35,6.090116,0.834492,0.027460,0.000000,0.000000,0.000000,0.000000,0.000000,0.506881,0.091535,1,0
3,1,1,17,16,16,14,12,9,75.438535,20.352500,5.237412,0.206817,0.003884,0.000971,0.000971,0.000971,0.544614,0.089329,1,1
4,1,1,63,63,63,59,57,48,13.558211,5.366467,0.604079,0.051511,0.000000,0.000000,0.000000,0.000000,0.552941,0.112387,0,1


In [7]:
# Convert the Class to factor
df['Class'] = df['Class'].astype('category')
df.dtypes

q             int64
ps            int64
nma.a         int64
nma.b         int64
nma.c         int64
nma.d         int64
nma.e         int64
nma.f         int64
nex.a       float64
nex.b       float64
nex.c       float64
nex.d       float64
nex.e       float64
nma.f.1     float64
nma.g       float64
nma.h       float64
dd          float64
dm          float64
amfm          int64
Class      category
dtype: object

In [8]:
# SPlit the dataset
x_train, x_test, y_train, y_test = train_test_split(df.values, df['Class'].values, 
                                                    test_size = 0.3, random_state=123)

#### Simple Ensemble Techniques

In [9]:
# Standardize the data
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(x_train)
scaler.mean_

array([9.93788820e-01, 9.11490683e-01, 3.88788820e+01, 3.72950311e+01,
       3.54332298e+01, 3.25015528e+01, 2.88819876e+01, 2.11428571e+01,
       6.29931750e+01, 2.26212520e+01, 8.37121561e+00, 1.79391036e+00,
       6.05581949e-01, 2.27242775e-01, 8.50907236e-02, 3.43097671e-02,
       5.22833116e-01, 1.08045463e-01, 3.30745342e-01, 5.45031056e-01])

In [10]:
scaler.scale_

array([7.85659046e-02, 2.84034184e-01, 2.51764134e+01, 2.36517810e+01,
       2.22842356e+01, 2.06703270e+01, 1.90111536e+01, 1.46116847e+01,
       5.81244675e+01, 2.13589826e+01, 1.13192798e+01, 4.23655140e+00,
       2.92335487e+00, 1.22486790e+00, 4.10876838e-01, 1.59464653e-01,
       2.87110822e-02, 1.73946414e-02, 4.70481520e-01, 4.97968075e-01])

In [11]:
# Transform the data
x_train_trans = scaler.transform(x_train)
x_test_trans = scaler.transform(x_test)

In [12]:
# 1. Max Voting
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= LogisticRegression()
model1.fit(x_train_trans,y_train)
model2.fit(x_train_trans,y_train)
model3.fit(x_train_trans,y_train)

pred1=model1.predict(x_test_trans)
pred2=model2.predict(x_test_trans)
pred3=model3.predict(x_test_trans)

final_pred = np.array([])
for i in range(0,len(x_test_trans)):
    final_pred = np.append(final_pred, mode([pred1[i], pred2[i], pred3[i]]))

In [13]:
trainCM1 = confusion_matrix(y_test, final_pred)
print('True Negative ='+ str(trainCM1[0,0]))
print('False Negative ='+ str(trainCM1[1,0]))
print('True Positive ='+ str(trainCM1[1,1]))
print('False Positive ='+ str(trainCM1[0,1]))

True Negative =145
False Negative =0
True Positive =131
False Positive =0


In [14]:
print('Testing Accuracy Percentage =',metrics.accuracy_score(final_pred, y_test)*100)
print('Training Precision =', trainCM1[1,1]/(trainCM1[1,1]+trainCM1[0,1])) #Precision=tp/(tp+fp)
print('Training Specificity =',trainCM1[0,0]/(trainCM1[0,0]+trainCM1[0,1]) )#True negative rate/Specificity=tn/(tn+fp)
print('Recall/True Positive Rate/Sensitivity =', trainCM1[1,1]/(trainCM1[1,1]+trainCM1[1,0])) #Recall=tp/(tp+fn)

Testing Accuracy Percentage = 100.0
Training Precision = 1.0
Training Specificity = 1.0
Recall/True Positive Rate/Sensitivity = 1.0


In [15]:
# 2. Averaging
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3= LogisticRegression()

model1.fit(x_train_trans,y_train)
model2.fit(x_train_trans,y_train)
model3.fit(x_train_trans,y_train)

pred1=model1.predict_proba(x_test_trans)[:,1]
pred2=model2.predict_proba(x_test_trans)[:,1]
pred3=model3.predict_proba(x_test_trans)[:,1]

finalpred=(pred1+pred2+pred3)/3
threshold = 0.5
preds = np.where(finalpred > threshold , 1,0)

In [16]:
trainCM2 = confusion_matrix(y_test, preds)
print('True Negative ='+ str(trainCM2[0,0]))
print('False Negative ='+ str(trainCM2[1,0]))
print('True Positive ='+ str(trainCM2[1,1]))
print('False Positive ='+ str(trainCM2[0,1]))

True Negative =145
False Negative =0
True Positive =131
False Positive =0


#### Advanced Ensemble Techniques

In [17]:
# 1. Blending
# SPlit Train/Validation/Test
X_train, X_test, Y_train, Y_test = train_test_split(df.values, df['Class'].values, 
                                                    test_size = 0.3, random_state=123)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1) 

X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)
X_val_trans=scaler.transform(X_val)

model1 = DecisionTreeClassifier()
model1.fit(X_train_trans, Y_train)
val_pred1=model1.predict(X_val_trans)
test_pred1=model1.predict(X_test_trans)
val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)

model2 = KNeighborsClassifier()
model2.fit(X_train_trans,Y_train)
val_pred2=model2.predict(X_val_trans)
test_pred2=model2.predict(X_test_trans)
val_pred2=pd.DataFrame(val_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [18]:
Xval = pd.DataFrame(X_val_trans)
Xtest = pd.DataFrame(X_test_trans)
df_val=pd.concat([Xval, val_pred1,val_pred2],axis=1)
df_test=pd.concat([Xtest, test_pred1,test_pred2],axis=1)

model = LogisticRegression()
model.fit(df_val,Y_val)
pred1=model.predict_proba(df_test)[:,1]
threshold = 0.5
prediction = np.where(pred1 > threshold , 1,0)

In [19]:
trainCM3 = confusion_matrix(Y_test, prediction)
print('True Negative ='+ str(trainCM3[0,0]))
print('False Negative ='+ str(trainCM3[1,0]))
print('True Positive ='+ str(trainCM3[1,1]))
print('False Positive ='+ str(trainCM3[0,1]))

True Negative =145
False Negative =0
True Positive =131
False Positive =0


In [20]:
print('Testing Accuracy Percentage =',metrics.accuracy_score(prediction, Y_test)*100)
print('Training Precision =', trainCM3[1,1]/(trainCM3[1,1]+trainCM3[0,1])) #Precision=tp/(tp+fp)
print('Training Specificity =',trainCM3[0,0]/(trainCM3[0,0]+trainCM3[0,1]) )#True negative rate/Specificity=tn/(tn+fp)
print('Recall/True Positive Rate/Sensitivity =', trainCM3[1,1]/(trainCM3[1,1]+trainCM3[1,0])) #Recall=tp/(tp+fn)

Testing Accuracy Percentage = 100.0
Training Precision = 1.0
Training Specificity = 1.0
Recall/True Positive Rate/Sensitivity = 1.0
